# Skip Lists, Commutative Hashing, and Authenticated Dictionaries

Goodrich and Tamassia's paper ["Efficient Authenticated Dictionaries with Skip Lists and Commutative Hashing"](http://www.cs.jhu.edu/~goodrich/cgc/pubs/hashskip.pdf) discuss a few interesting concepts.

The Goodrich paper uses the skip list data structure and a concept called _commutative hashing_ to build an _authenticated dictionary_. The described use-case of an authenticated dictionary is a certificate revocation statusing system. In this example a certificate authority (CA) is an authoratative source of a set of certificates and their revocation status. Instead of relying on querying the CA for trustworthy revocation checks, entities interested in revocation status can query other (untrusted) services for revocation status and a non-interactive proof of status from the CA.

**TODO(kkl)**: Add a diagram of Checker -> Untrusted Service -> CA.

**TODO(kkl)**: Diagrams of Plateau and Tower elements.

## An overview of terminology used in the paper

$h$ - A commutative hash function

$S$ - A set of elements for which we want to enable authenticated membership queries.

$S_t$ - A linked list at level $t$. $S_0$ is the base list that contains all elements of $S$ in order.

$v$ - A node within a linked list $S_t$.

$s$ - A "start" node when traversing a skip list.

$f(v)$ - A "label" computed of a value $v$ that computed from $h$ and the element's position in a skip list.

$f(s)$ - The label of the start node.

$elem(v)$ - The element stored at the node $v$. E.g. The number 32.

"plateau" - The top-most node of a "column" in a skip list. 

"tower" - Any node that isn't a plateau node.


## Skip lists

Skip lists are a data structure that were introduced by [William Pugh](https://epaperpress.com/sortsearch/download/skiplist.pdf) as a (simpler) to implement alternative to self-balancing trees with reasonably efficient search, insertion, and deletion properties.

For demonstration purposes, let's build an incomplete version of a skip list and some helper functions. You can skim the details here if you are familar with skip lists.

For clarity, I'll stick with naming conventions used in Goodrich's paper. First, let's build a "Node" element which will store integer values (i.e. "elements" in Goodrich's paper) and pointers to other Nodes.

In [29]:
import random
import hashlib

# Sentinel values for the beginning and end of linked lists
INF = float('inf')
NINF = -INF


class Node(object):
    """Node wraps element values and pointers to other nodes.
    
    Most skiplists only have right and down pointers. For helper 
    functions like `is_plateau` implementation is more obviously 
    correct if we maintain up pointers too.
    
    Similarly, skiplist nodes don't usually need to know what level 
    they are on. Keeping a level reference for debugging purposes
    is nice though.
    """
    def __init__(self, elem):
        assert type(elem) is int or elem in (INF, NINF)
        self.elem = elem
        self.level = None
        self.right = None
        self.up = None
        self.down = None
        
    def __str__(self):
        return str(self.elem)
    
    def __eq__(self, other):
        return self.elem == other.elem
    
    def __lt__(self, other):
        return self.elem < other.elem
    
    def __gt__(self, other):
        return self.elem > other.elem
    
    def __le__(self, other):
        return self.elem <= other.elem
    
    def __ge__(self, other):
        return self.elem >= other.elem
        
    def _debug(self, f):
        return "Node{level: %s, elem:%s, label:%s}" % (self.level, self.elem, f(self))
    
    @property
    def is_plateau(self):
        """A boolean indicating whether the node is a "plateau" node.
        
        The definition of a plateau node can be found in Section 2.1 
        of Goodrich's paper.
        """
        return self.up is None
    
    @property
    def is_tower(self):
        """A boolean indicating whether the node is a "tower" node.
        
        The definition of a tower node can be found in Section 2.1 
        of Goodrich's paper.
        """
        return not self.is_plateau
    

def NegInf():
    """Returns a Node with element value -Infinity.
    """
    return Node(NINF)


def Inf():
    """Returns a Node with element value +Infinity.
    """
    return Node(INF)


# A few basic test cases
assert Node(1) == Node(1)
assert Node(2) >= Node(1)
assert Node(1) <= Node(2)
assert Node(2) > Node(1)
assert Node(1) < Node(2)
assert NegInf() <= Node(1)
assert Inf() > Node(1)

Now let's build a partially complete skip list. This skip list implementation doesn't support a few things like deletions, a dynamic number of levels, nor am I certain it will handle duplicates correctly. But for demonstration purposes, it should suffice.

The implementation details are not critical, so feel free to skim over them.

In [30]:
def random_level(rng, max_level):
    """Determine what levels of the skip list a node should be 
    inserted in, capped at `max_level`.
    """
    level = 1
    while rng.randint(0, 1):
        level += 1
    return min(max_level, level)

class SkipList(object):
    """A partially implemented skip list.
    """
    def __init__(self, *, height=1):
        assert height >= 1
        self.rng = random.Random(11)
        self.height = height
        # levels are zero-indexed so subtract one
        self.max_level = height-1
        
        # build initial levels containing only -inf and inf
        # start with highest level first, out of the loop
        self.start = NegInf()
        self.start.right = Inf()
        
        self.start.level = self.max_level
        self.start.right.level = self.max_level
        
        # build remaining levels from top-to-bottom, skipping the
        # topmost level as we just init'd that one.
        last_ninf = self.start
        last_inf = self.start.right
        for level in reversed(range(self.max_level)):
            # Create new -inf/inf nodes
            ninf = NegInf()
            inf = Inf()
                  
            # Add right pointer 
            ninf.right = inf
        
            # Add levels
            ninf.level = level
            inf.level = level
            
            # Add up/down pointers
            last_ninf.down = ninf
            last_inf.down = inf
            ninf.up = last_ninf
            inf.up = last_inf
            
            # update last_{n}inf
            last_ninf = ninf
            last_inf = inf

    def __str__(self):
        return " ".join(str(e) for e in self.level(0))
    
    def _debug(self, f):
        s = "SkipList{\n"
        for i in reversed(range(self.height)):
            s += "  %d: " % i
            s += " ".join(n._debug(f) for n in self._level(i))
            s += "\n"
        s += "}"
        return s
    
    def _level(self, level_index):
        """Return the nodes at the specific level_index as a Python list
        """
        if level_index < 0 or level_index > self.max_level:
            raise IndexError('level_index %d out of bounds' % level_index)
        curr = self.start
        nodes = []
        for _ in range(self.height - level_index - 1):
            curr = curr.down  
        while curr is not None:
            nodes.append(curr)
            curr = curr.right
        return nodes
            
    def level(self, level_index):
        """Return the elems at the specific level_index as a Python list
        """
        return [n.elem for n in self._level(level_index)]
    
    def insert(self, elem):
        """Insert a Node with elem into the skiplist.
        """
        # The current node position
        curr = self.start
        # The level of the current node
        currLevel = self.height
        # The randomly assigned levels we will insert at
        insertLevel = random_level(self.rng, self.max_level)
        # A reference to the last node we inserted, used to add down pointers
        lastNode = None
        while currLevel > 0:
            # A few conditions we'll need to determine what to do at this level
            rightIsNone = curr.right is None
            rightIsLarger = rightIsNone or curr.right > Node(elem)
            insertAtCurrentLevel = insertLevel >= currLevel
            
            if not rightIsLarger:
                # The node to the right is not larger, move right.
                curr = curr.right
                continue
            
            if rightIsLarger and insertAtCurrentLevel:
                # The node to the right is larger and we insert at this level.
                # First, we keep a reference to the node that was previously
                # right of the current node
                oldRight = curr.right
                
                newNode = Node(elem)
                newNode.level = currLevel-1
                
                # Replace right with the new Node.
                # The new node's right node is the old right node
                curr.right = newNode
                newNode.right = oldRight
                
                # If we have previously inserted a node at a different level,
                # add up/down pointers to the previously inserted node and 
                # new node.
                if lastNode is not None:
                    lastNode.down = newNode
                    newNode.up = lastNode
                    
                lastNode = newNode

            # Move down a level.
            currLevel -= 1
            curr = curr.down
    
    def contains(self, elem):
        """Returns true/false on whether a node with value elem is in the skip list.
        """
        if self.get(elem) is None:
            return False
        return True
        
    def get(self, elem):
        """Returns the top-most element with value elem is in the skip list, or None.
        """
        curr = self.start
        node = Node(elem)
        while curr is not None:
            right = curr.right
            if right is None:
                curr = curr.down
                continue
            if node < right:
                curr = curr.down
                continue
            if node > right:
                curr = right
                continue
            if node == right:
                return right
        return None

To get a feel for the `SkipList` API (and to test it's mostly functional), let's insert some elements and search for elements.

In [55]:
# Build a skip list and insert a few elements
skip_list = SkipList(height=3)
skip_list.insert(3)
skip_list.insert(2)
skip_list.insert(1)

# Assertions to make sure SkipList isn't horribly borked
assert skip_list.height == 3
assert skip_list.start.up is None
assert skip_list.max_level == 2
assert [NINF, 1, 2, 3, INF] == skip_list.level(0)
assert skip_list.contains(3)
assert not skip_list.contains(-1)

# `get` should always return the top-most element
for i in [1, 2, 3]:
    assert skip_list.get(i).is_plateau 

## Commutative Hashing

Commutative hash functions are a relaxation of typical collision resistant hash function. That is, a commutative hash function $h(x, y)$ is defined to return the same results regardless of the order of the inputs, so there is a trivial case that would normally be considered a collision but is not: $h(x, y) = h(y, x)$. Commutative hash function collisions explicitly call out this collision as irrelevant.

**TODO: Write about the value-add of commutative hashing in this context**

> Note that our scheme requires only the repeated accumulation of a sequence of values with a hash function. Unlike the previous best hash tree schemes, there is no need to provide auxiliary information about the order of the arguments to be hashed at each step, as determined by the topology of the path in a hash tree.

The commutative hash function they implement is simple. We first hash the smaller value of the two inputs and then the larger. This is implemented below. 

In [56]:
def _to_bytes(i):
    """Serialize a number into a byte string.
    
    Assumes input will always be a 256-bit integer. Sentinel values
    -Inf/Inf are serialized using values that are not otherwise
    possible under this assumption.
    """
    if i == INF:
        return b"\xff" * ((256//8)+1)
    elif i == NINF:
        return b"\x00" * ((256//8)+1)
    else:
        return i.to_bytes(256//8, byteorder='big')

def h(x, y):
    """Commutative hashing algorithm as described in Goodrich section 3.2.
    
    This function truncates the output of SHA-256 for readability 
    but it is not required.
    """
    # The Goodirch paper doesn't explicitly state how to handle the -Infinity
    # case but there are cases where it expects us to hash -Infinity elements.
    # E.g. The bottom-left node will hit case (1a) which requires
    # a hash of elem(v).
    fst, sec = min(x, y), max(x, y)
    sha256 = hashlib.sha256()
    sha256.update(_to_bytes(fst))
    sha256.update(_to_bytes(sec))
    return int.from_bytes(sha256.digest(), byteorder='big') % 0xff

assert h(1, 2) == h(2, 1)
assert h(1, 2) != h(2, 3)

## Hashing the skip list

Now that we have a skip list implementation, we can build the "label" function described in Goodrich 4.1. This label is derived from the values to the right and below a given node $v$ and the commutative hash function $h$ we just implemented.

This function, given a node in the skip list, returns it's label.

In [57]:
def _f(v, stack):
    w = v.right
    u = v.down
    # Base case: w/right is None. Default to a zero value.
    if w is None:
        return 0, stack
    
    # Case 1: We are at the bottom level of the skiplist.
    if u is None:
        # Case 1a
        if w.is_tower:
            return h(v.elem, w.elem), stack
        # Case 1b
        _fw, stack = _f(w, stack)
        hsh =  h(v.elem, _fw)
        return hsh, stack
    else:
        # Case 2a
        if w.is_tower:
            _fu, stack = _f(u, stack)
            return _fu, stack
        # Case 2b
        _fu, stack = _f(u, stack)
        _fw, stack = _f(w, stack)
        hsh = h(_fu, _fw)
        return hsh, stack

def f(v):
    """The "label" function described in Goodrich 4.1.
    """
    h, _ = _f(v, [])
    return h

    
start = skip_list.start
assert f(start) == f(start)

print(skip_list._debug(f))

SkipList{
  2: Node{level: 2, elem:-inf, label:205} Node{level: 2, elem:inf, label:0}
  1: Node{level: 1, elem:-inf, label:208} Node{level: 1, elem:1, label:25} Node{level: 1, elem:3, label:115} Node{level: 1, elem:inf, label:0}
  0: Node{level: 0, elem:-inf, label:61} Node{level: 0, elem:1, label:69} Node{level: 0, elem:2, label:159} Node{level: 0, elem:3, label:115} Node{level: 0, elem:inf, label:0}
}


# Notes

## Duplicates in a skip list

The paper makes no mention of duplicate values in the skip list.

**TODO(kkl): Test problems with duplicates?**

## h(-inf, y)

The paper doesn't explicitly mention that you need to handle hashing the element value of `-inf`. Specifically, take the bottom left element of a skip list to be $v$. When applying the label function $f$ (Described in section 4.1) to $v$ we will hit cases 1a/1b (dependent on the node right of $v$). In both cases, we compute $h(elem(v), ...)$ where $elem(v)$ is `-inf`.

Since I used SHA256 as the hash function (The authors mentioned they used MD5 in their paper which would suffer from the same problem) one would have to design a mapping from `-inf` to hashable byte strings. For implementation simplicity, I chose to just convert `-inf` cases to 0.

This is a problem, as this introduces collisions! E.g. h(-inf, 0) == h(0, 0). **TODO(kkl): Demonstrate this**.

## plateau vs. tower

I added `up` pointers which aren't totally necesary. When traversing the skip list any moves `right` will land you on a plateau.